In [29]:
from typing import List, Union, Callable, Tuple
import numpy as np
import pandas as pd

In [30]:
df = pd.DataFrame({'key1': ['a', 'a', None, 'b', 'b', 'a', None],
                  'key2': pd.Series([1, 2, 1, 2, 1, None, 1], dtype='Int64'),
                  'data1': np.random.standard_normal(7),
                  'data2': np.random.standard_normal(7)})

In [31]:
df

,key1,key2,data1,data2
0,a,1,0.142701,-0.589421
1,a,2,0.051929,0.621849
2,None,1,0.410200,-2.031932
3,b,2,0.783424,-0.315132
4,b,1,0.891629,-0.594915
5,a,<NA>,0.069160,0.168668
6,None,1,0.593394,-0.650341


In [32]:
grouped = df.groupby('key1')

In [33]:
grouped

In [34]:
grouped['data1'].nsmallest(2)

key1   
a     1    0.051929
      5    0.069160
b     3    0.783424
      4    0.891629
Name: data1, dtype: float64

In [35]:
#to use one's own aggregate function: pass any function aggregating any array to the aggregate method (alias agg).
grouped['data1'].nsmallest(2)

key1   
a     1    0.051929
      5    0.069160
b     3    0.783424
      4    0.891629
Name: data1, dtype: float64

In [36]:
grouped['data1'].nlargest(2)

key1   
a     0    0.142701
      5    0.069160
b     4    0.891629
      3    0.783424
Name: data1, dtype: float64

In [37]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [38]:
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,0.090772,1.211270
b,1,0.108205,0.279783


In [39]:
#you may notice that some methods also work even if they are not aggregations
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.087930  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.837526  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.105931  0.142701   3.0  0.067032  0.611998 -0.589421 -0.210376   
b     0.864578  0.891629   2.0 -0.455023  0.197836 -0.594915 -0.524969   

                                    
           50%       75%       max  
key1                                
a     0.168668  0.395259  0.621849  
b    -0.455023 -0.385078 -0.315132  

[2 rows x 24 columns]

In [40]:
#column-wise and multiple function application
#case study: Tips dataset
tips = pd.read_csv('examples/tips.csv')

In [41]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [42]:
#now add a tip_pct column
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [43]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [44]:
#group tips by "day" and "smoker"
grouped = tips.groupby(['day', 'smoker'])

In [45]:
#note for descriptive statistics like in table 10-1, we can pass the function as a string
grouped_pct = grouped['tip_pct']

In [46]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [47]:
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [48]:
#To implement a list of functions or function names, then we get back a DataFrame with column
#names taken from the functions
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [49]:
#To specify names for functions: e.g. with lambda functions: pass a list of tuples.
#Item 0 becomes the name.
#Item 1 becomes the description.
grouped_pct.agg([('average', 'mean'), ('stdev', np.std)])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [50]:
#DataFrames
#We can apply the same function to all of the columns or different functions per column.
#Example: Let's compute three statistics the tip_pct and total_bill columns: 1) count 2) mean 3) max.
functions : List[Union[str, Callable]] = ['count', 'mean', 'max']

In [51]:
result = grouped[['tip_pct', 'total_bill']].agg(functions)

In [52]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [53]:
#resulting dataframe uses heirarchical columns, just as if we aggregated separately and grouped using
#the column names as the keys argument
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [56]:
#we can also pass a list of tuples with custom names
ftuples : List[Tuple[str, Union[str, Callable]]] = [('Average', 'mean'), ('Variance', np.var)]
ftuples

[('Average', 'mean'),
 ('Variance',
  <function numpy.var(a, axis=None, dtype=None, out=None, ddof=0, keepdims=<no value>, *, where=<no value>)>)]

In [57]:
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518